# _Mini Program - Working with SQLLite DB using Python_

### <font color=green>Objective -</font>
<font color=blue>1. This program gives an idea how to connect with SQLLite DB using Python and perform data manipulation </font><br>

<font color=blue>2. There are 2 ways in which tables are create below to help you understand the robustness of this language</font>

### <font color=green>Step 1 - Import required libraries</font>
#### <font color=blue>In this program we make used of 3 libraries</font>
#### <font color=blue>1. sqlite3 - This module help to work with sql interface. It helps in performing db operations in sqllite database</font>
#### <font color=blue>2. pandas - This module provides high performance and easy to use data manipulation and data analysis functionalities</font>
#### <font color=blue>3. os - This module provides function to interact with operating system with easy use</font>
    


In [1]:
#Importing the required modules
import sqlite3
import pandas as pd
import os

### <font color=green>Step 2 - Creating a function to drop the table</font>
#### <font color=blue>Function helps to re-create a reusable component that can be used conviniently and easily in other part of the code</font>
#### <font color=blue>In Line 1 - We state the function name and specify the parameter being passed. In this case, the parameter is the table name</font>
#### <font color=blue>In Line 2 - We write the sql query to be executed</font>
#### <font color=blue>In Line 3 - We execute the query using the cursor object</font>

In [2]:
#Creating a function to drop the table if it exists
def dropTbl(tablename):
    dropTblStmt = "DROP TABLE IF EXISTS " + tablename
    c.execute(dropTblStmt)

### <font color=green>Step 3 - We create the database in which our table will reside</font>
#### <font color=blue>In Line 1 - We are removing the already existing database file</font>
#### <font color=blue>In Line 2 - We use connect function from the sqlite3 module to create a database studentGrades.db and establish a connection</font>
#### <font color=blue>In Line 3 - We create a context of the database connection. This help to run all the database queries</font>

In [3]:
#Removing the database file
os.remove('studentGrades.db')

#Creating a new database - studentGrades.db
conn = sqlite3.connect("studentGrades.db")
c = conn.cursor()

### <font color=green>Step 4 - We create a table in sqllite DB using data defined in the excel file</font>
#### <font color=blue>This is the first method in which you can create a table. You can use to_sql function directly to read a dataframe and dump all it's content to the table</font>
#### <font color=blue>In Line 1 - We are making use of dropTbl function created above to drop the table</font>
#### <font color=blue>In Line 2 - We are creating a dataframe from the data read from the csv</font>
#### <font color=blue>In Line 3 - We use to_sql function to push the data into the table. The first row of the file becomes the column name of the tables</font>
#### <font color=blue>We repeat the above steps for all the 3 files to create 3 tables - STUDENT, GRADES and SUBJECTS</font>

In [4]:
#Reading data from csv file - student details, grades and subject
dropTbl('STUDENT')
student_details = pd.read_csv("Datafiles/studentDetails.csv")
student_details.to_sql('STUDENT',conn,index = False)

dropTbl('GRADES')
student_grades = pd.read_csv('Datafiles/studentGrades.csv')
student_grades.to_sql('GRADES',conn,index = False)

dropTbl('SUBJECTS')
subjects = pd.read_csv("Datafiles/subjects.csv")
subjects.to_sql('SUBJECTS',conn,index = False)

### <font color=green>Step 5 - We create a master table STUDENT_GRADE_MASTER where we can colate the data from the individual tables by performing the joining operations</font>
#### <font color=blue>In Line 1 - We are making use of dropTbl function created above to drop the table</font>
#### <font color=blue>In Line 2 - We are writing sql query for table creation</font>
#### <font color=blue>In Line 3 - We are using the cursor created above to execute the sql statement</font>
#### <font color=blue>In Line 4 - We are using the second method of inserting data into the table. We are writing a query to insert the data after joining the data from all the tables</font>
#### <font color=blue>In Line 5 - We are using the cursor created above to execute the sql statement</font>
#### <font color=blue>In Line 6 - We are doing a commit operation. Since INSERT operation is a ddl, we have to perform a commit operation to register it into the database</font>

In [5]:
#Creating a table to store student master data
dropTbl('STUDENT_GRADE_MASTER')
createTblStmt = '''CREATE TABLE STUDENT_GRADE_MASTER
                    ([Roll_number] INTEGER,
                     [Student_Name] TEXT,
                     [Stream] TEXT,
                     [Subject] TEXT,
                     [Marks] INTEGER
                    )'''
c.execute(createTblStmt)

#Inserting data into the master table by joining the tables mentioned above
queryMaster = '''INSERT INTO STUDENT_GRADE_MASTER(Roll_number,Student_Name,Stream,Subject,Marks)
                 SELECT g.roll_number, s.student_name, stream, sub.subject, marks from GRADES g 
                 LEFT OUTER JOIN STUDENT s on g.roll_number = s.roll_number
                 LEFT OUTER JOIN SUBJECTS sub on g.subject_code = sub.subject_code'''

c.execute(queryMaster)
c.execute("COMMIT")

### <font color=green>Step 6 - We can perform data fetch like we do in sqls using this sqlite3 module</font>
#### <font color=blue>In Line 1 - We are writing a query to find the number of records in the master table</font>
#### <font color=blue>In Line 2 - We are executing the above created query</font>
#### <font color=blue>In Line 3 - fetchall function is used to get the result returned by the query. The result will be in the form of a list of tuples</font>
#### <font color=blue>In Line 4 - We are writing another query to find the maximum marks recorded for each subject</font>
#### <font color=blue>In Line 5 - We are executing the above created query</font>
#### <font color=blue>In Line 6 - fetchall function is used to get the result returned by the query. The result will be in the form of a list of tuples</font>
#### <font color=blue>In Line 7 - We are writing another query to find the percentage of marks obtained by each student in the class</font>
#### <font color=blue>In Line 8 - We are executing the above created query</font>
#### <font color=blue>In Line 9 - fetchall function is used to get the result returned by the query. The result will be in the form of a list of tuples</font>

In [6]:
#Finding the key data from the master table

#1. Find the number of records in the master table
query_count = '''SELECT COUNT(*) FROM  STUDENT_GRADE_MASTER'''
c.execute(query_count)
number_of_records = c.fetchall()
print(number_of_records)

#2. Maximum marks for each subject
query_max_marks = '''SELECT Subject,max(Marks) as 'Max_Marks' from STUDENT_GRADE_MASTER GROUP BY Subject'''
c.execute(query_max_marks)
max_marks_data = c.fetchall()
print(max_marks_data)

#3. Percenatge of marks scored by each student
query_percentage = '''SELECT Student_Name, avg(Marks) as 'Percentage' from STUDENT_GRADE_MASTER GROUP BY Student_Name'''
c.execute(query_percentage)
percentage_data = c.fetchall()
print(percentage_data)


[(20,)]
[('C', 97), ('C++', 95), ('Environmental studies', 92), ('Java', 96), ('Maths', 98)]
[('Abhishek', 94.2), ('Anand', 85.2), ('Sourabh', 89.0), ('Vivek', 84.8)]


### <font color=green>Step 7 - We are closing the database connection</font>
#### <font color=blue>It is always a good practice to close the database connection after all the operations are completed</font>

In [7]:
#Closing the connection
conn.close()